## Run in blender

In [6]:
import bpy
import glob

vrm_filepath = "/Users/pataranansethpakdee/Documents/GitHub/3D-Waifu-Model-Generator/Datasets/3D_VRMModel/"
processed_filepath = "/Users/pataranansethpakdee/Documents/GitHub/3D-Waifu-Model-Generator/Datasets/3D_ProcessedModel/"
image_filepath = "/Users/pataranansethpakdee/Documents/GitHub/3D-Waifu-Model-Generator/Datasets/2D_Image/"
vrm_files = glob.glob(f'{vrm_filepath}*')

def purge_orphans():
    bpy.ops.outliner.orphans_purge(do_local_ids=True, do_linked_ids=True, do_recursive=True)

def clean_scene():
    scene = bpy.context.scene
    bpy.data.scenes.new("Scene")
    bpy.data.scenes.remove(scene, do_unlink=True)
    purge_orphans()
    
def set_new_camera():
    scene = bpy.context.scene
    cam_data = bpy.data.cameras.new(name="Camera")
    cam = bpy.data.objects.new(name="Camera", object_data=cam_data)
    scene.collection.objects.link(cam)
    scene.camera = cam
    cam.location = (0, -5.3, 0.8)
    cam.rotation_euler = (1.5708, 0, 0) 

def new_scene():
    clean_scene()
    set_new_camera()

count = 0

for file in vrm_files:
    
    file_name = str(count).zfill(4)

    clean_scene()
    set_new_camera()
    bpy.ops.import_scene.vrm(filepath=file)
    scene = bpy.context.scene


    # Render settings
    scene.render.filepath = f'{image_filepath}{file_name}.png'
    scene.render.image_settings.file_format = 'PNG'
    scene.render.resolution_x = 1920
    scene.render.resolution_y = 1080

    # Render the image
    bpy.ops.render.render(write_still=True)

    bpy.ops.wm.ply_export(filepath=f"{processed_filepath}{file_name}.ply")

    count+=1



# Preprocessing

In [10]:
#Convert .ply to point cloud
import open3d as o3d

# Load the 3D model
mesh = o3d.io.read_triangle_mesh("Datasets/3D_ProcessedModel/0000.ply")

# Sample points from the mesh surface
pcd = mesh.sample_points_uniformly(number_of_points=100000)

# Save the point cloud
o3d.io.write_point_cloud("Processed_Data/3D_PointCloud/point_cloud.ply", pcd)


True